In [4]:
#!pip install -e .
#!pip uninstall sqlalchemy -y
#!pip install sqlalchemy==1.4.51
!pip install --upgrade langchain langchain-community

  Using cached langchain-0.2.10-py3-none-any.whl.metadata (6.9 kB)
  Using cached langchain_core-0.2.22-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
Using cached langchain-0.2.10-py3-none-any.whl (990 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.5 MB/s eta 0:00:00:00:01
Using cached langchain_core-0.2.22-py3-none-any.whl (373 kB)
Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.20
    Uninstalling

In [ ]:
from transformers import pipeline
from datasets import load_dataset
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# Load the HRMIS_MASTER dataset
logging.info("Loading dataset...")
dataset = load_dataset("Kaballas/HRMIS_MASTER")

# Initialize the question generator
logging.info("Initializing question generator...")
question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

def generate_question(questions, answer):
    input_text = f"answer: {answer}  questions: {questions}"
    #logging.info(f"Generating question for questions: {questions[:30]}... and answer: {answer}")
    generated = question_generator(input_text, max_length=64, num_return_sequences=1)
    logging.info(f"Generated question: {generated[0]['generated_text']}")
    return generated[0]['generated_text']

augmented_pairs = []

# Assuming the dataset has 'questions' and 'answer' fields
logging.info("Generating augmented pairs...")
for item in dataset['train']:  # or whichever split you want to use
    questions = item['questions']
    answer = item['answers']
    
    # Original pair
    augmented_pairs.append((questions, answer))
    
    # Generate a new question
    #new_question = generate_question(questions, answer)
    #logging.info(f"new_question pairs generated. {new_question}")
    #augmented_pairs.append((new_question, questions))

logging.info("Augmented pairs generated.")

# Initialize RAGTrainer
from ragatouille import RAGTrainer

logging.info("Initializing RAGTrainer...")
trainer = RAGTrainer(model_name="HRMIS_QuestionGeneratedColBERT", pretrained_model_name="mixedbread-ai/mxbai-colbert-v1")

logging.info("Preparing training data...")
trainer.prepare_training_data(raw_data=augmented_pairs, data_out_path="./hrmis_question_generated_data/")
logging.info("Training data prepared.")


INFO:root:Loading dataset...
INFO:root:Initializing question generator...
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:root:Generating augmented pairs...
INFO:root:Augmented pairs generated.
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:root:Initializing RAGTrainer...
INFO:root:Preparing training data...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


Loading Hard Negative SimpleMiner dense embedding model BAAI/bge-small-en-v1.5...


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:CUDA/NPU is not available. Starting 4 CPU workers
INFO:sentence_transformers.SentenceTransformer:Start multi-process pool on devices: cpu, cpu, cpu, cpu
